OECD API with Python
======

## Example showing GDP growth rates by selected country

-----

*October 17, 2018*<br>
*@bd_econ*

The documentation for the Organization for Economic Cooperation and Development (OECD) API can be found [here](https://data.oecd.org/api/)

In [1]:
import requests
import pandas as pd

## Parameters/ Settings

In [2]:
base = 'http://stats.oecd.org/sdmx-json/data/'
param = [('dataset', 'QNA'),
         ('country', 'FRA+ITA+ESP+GBR+CAN+DEU'),
         ('indicators', 'GDP+B1_GE.CUR+VOBARSA'), 
         ('freq', 'Q'), 
         ('start_date' , '?startTime=1999-Q4')
        ]

series = '.'.join(x[1] for x in param[1:-1])
url = '{}{}/{}{}'.format(base, param[0][1], series, param[-1][1])

## Request data

In [3]:
r = requests.get(url).json()

In [4]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = pd.to_datetime([x['id'] for x in date_list])
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][0]['values']]

title = r['structure']['dimensions']['series'][1]['values'][0]['name']

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '{}:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
    df[area] = (((df[area]/df[area].shift())**4)-1)*100
df.index = dates
df = df.dropna()
print(title)
df.tail().round(1)

Gross domestic product - expenditure approach


Canada  France  Germany  Italy  Spain  United Kingdom
2019-01-01     1.2     2.2      2.5    0.8    2.3             2.7
2019-04-01     3.2     1.0     -2.0    0.4    1.5            -0.2
2019-07-01     1.1     0.6      1.2    0.0    1.6             2.1
2019-10-01     0.6    -0.7     -0.1   -0.9    1.7            -0.0
2020-01-01    -8.2   -21.7     -7.8  -19.8  -19.3            -8.5